### Title: Model Combination and Encompassing

Author: Yiran Jing

Date: 31-10-2018

##### Summary

- Attempting to combine AR(336) and ridge model (second best and third best model)
- Evaluating whether is combination of models useful with forecast encompassing.


##### Result
**Must mention the following points below, as we should try all reasonable/possible combination, but some of them didnot work well**
- AR(336) and ridge combination seems to have improved our forecasts.
- We tried combine NN with AR or ridge as well, but cannot get lower MAPE than signle NN. 
- We also tried combine NN, Ridge and AR three model together, but cannot imporve MAPE as well

In [282]:
from collections import Counter
import pandas as pd
import csv
import pprint
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from collections import defaultdict
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib as mpl
from pandas import Series
from matplotlib import pyplot
import statsmodels.api as sm
import seaborn as sns
import datetime
import glob, os
from statsmodels.tsa.arima_model import ARIMA
import forecast
import warnings
warnings.filterwarnings('ignore')

# Plot settings
sns.set_context('notebook') 
sns.set_style('ticks')
red='#D62728'
blue='#1F77B4'
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

### Introduction

https://labs.eleks.com/2016/10/combined-different-methods-create-advanced-time-series-prediction.html

Combining forecasts have been shown in the literature to greatly improve forecasts and also intuitively makes sense as well.

Recall that if we had a 1-step ahead forecast from 2 different models, which we denote as $\hat{y}_{t+1}^{(1)}$ and $\hat{y}_{t+1}^{(2)}$. Furthermore, we can compute the forecast errors which we denote as $e_{t+1}^(1)$ and $e_{t+1}^(2)$. From this, letting $\lambda$ be the weight parameter, we denote the combined forecast as 

$$
\hat{y}_{t+1}^{c} = (1-\lambda)\hat{y}_{t+1}^{(1)} + \lambda \hat{y}_{t+1}^{(2)}
$$

From this, the variance of the combined forecast error is 

$$
Var(e_{t+1}^c) = (1 - \lambda)^2\sigma_1^2 + \lambda^2\sigma_2^2 + 2\lambda(1-\lambda)\rho \sigma_1\sigma_2
$$

We can then optimise $\lambda$ to minimise the variance, which gives us 
$$
\lambda^* = \frac{\sigma_1^2 - \rho \sigma_1 \sigma_2}{\sigma_1^2 + \sigma_2^2 - 2\rho \sigma_1\sigma_2}
$$

However, we need to use estimates so we have that

$$
\hat{\lambda^*} = \frac{\hat{\sigma}_1^2 -  \hat{\sigma}_{12}}{\hat{\sigma}_1^2 + \hat{\sigma}_2^2 - 2\hat{\sigma}_{12}}.
$$

where we use $\hat{\sigma}$ as an estimator for $\sigma$,to be the residuals in our estimated model.

In [227]:
data = pd.read_csv("../data/combine_model_result_10_29.csv")
del data['period.1']

### AR(336)

In [69]:
ar_res = data['demand'] - data['ar_336_fit']
ar_res = ar_res[:12910]

In [289]:
ar_var = np.var(ar_res)
ar_var

3848.608502066189

### Ridge model

In [290]:
ridge_res = data['demand'] - data['ridge_fit']
ridge_res = ridge_res[:12910]
ridge_var = np.var(ridge_res)
ridge_var

4963.336670554788

### NN model

In [291]:
nn_res = data['demand'] - data['nn_fit']
nn_res = nn_res[:12910]
nn_var = np.var(nn_res)
nn_var

3611.1306860790373

### Combine NN and Ridge 
**didnot work well , can ignore it**

As Variance of NN and AR are simiailar, and much lower tna. Ridge, so we combine AR and NN first to see result!

In [292]:
print("Variance of forecast for AR(336) is {} and variance of nn is {}".format(ar_var, nn_var))

Variance of forecast for AR(336) is 3848.608502066189 and variance of nn is 3611.1306860790373


In [293]:
# Get the 1,2 entry to get the covariance from covariance matrix.
cov = np.cov(ar_res, nn_res)[0][1]
print("Covariance of residuals is {}".format(cov))

Covariance of residuals is -2978.011957322873


In [294]:
opt_lambda = (nn_var - cov)/(ar_var + nn_var - (2*cov))
print("Lambda to use is {}".format(opt_lambda))

Lambda to use is 0.49114929899374316


** Hence, we should assign 49% of forecast for NN whilst we assign 51% to AR(336) model **

In [295]:
# Get new weighted forecast.
combined_forecast = opt_lambda*data['ar_336_pred'][12910:] + (1 - opt_lambda)* data['nn_pred'][12910:]

In [296]:
def mape_no_cv(y_pred, y):
    mape = np.abs((y-y_pred))/np.abs(y) # Check this definition
    return np.mean(mape)
    
def mae_no_cv(y_pred, y):
    return mean_absolute_error(y, y_pred) # Check the order of the inputs here

In [297]:
MAPE_combined = mape_no_cv(combined_forecast, data['demand'][12910:])
MAE_combined = mae_no_cv(combined_forecast, data['demand'][12910:])

In [298]:
MAPE_combined

0.015413922866349334

In [299]:
MAE_combined

19.900599783822358

### Combine AR and RIdge:
**work well! write this combination in report!!! **


In [300]:
print("Variance of forecast for AR(336) is {} and variance of ridge is {}".format(ar_var, ridge_var))

# Get the 1,2 entry to get the covariance from covariance matrix.
cov = np.cov(ar_res, ridge_res)[0][1]
print("Covariance of residuals is {}".format(cov))

opt_lambda = (ar_var - cov)/(ar_var + ridge_var - (2*cov))
print("Lambda to use is {}".format(opt_lambda))

# Get new weighted forecast.
combined_forecast = opt_lambda*data['ar_336_pred'][12910:] + (1 - opt_lambda)* data['ridge_pred'][12910:]

MAPE_combined = mape_no_cv(combined_forecast, data['demand'][12910:])
MAE_combined = mae_no_cv(combined_forecast, data['demand'][12910:])

Variance of forecast for AR(336) is 3848.608502066189 and variance of ridge is 4963.336670554788
Covariance of residuals is -2270.6642819502667
Lambda to use is 0.45826011693897234


In [301]:
MAPE_combined

0.016086785518219297

In [302]:
MAE_combined

20.7334753665034

**Very good MAPE improvement! compared with the single ridge or ar model! **

### Run linear regression
**not better resullt! **

In [229]:
import statsmodels.formula.api as smf
formula = "demand ~ ar_336_fit + nn_fit + ar_48_fit+ ridge_fit"
model_2 = smf.ols(formula = formula, data = data[:12910]).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                 demand   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.007e+06
Date:                Mon, 29 Oct 2018   Prob (F-statistic):               0.00
Time:                        01:22:01   Log-Likelihood:                -55149.
No. Observations:               12910   AIC:                         1.103e+05
Df Residuals:                   12905   BIC:                         1.103e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     18.8543      0.689     27.378      0.0

In [230]:
# Get new weighted forecast.
combined_forecast = model_2.params['ar_336_fit']/sum(model_2.params[1:])*data['ar_336_pred'][12910:] + \
                    model_2.params['nn_fit']/sum(model_2.params[1:])* data['nn_pred'][12910:]+\
                    model_2.params['ridge_fit']/sum(model_2.params[1:])* data['ridge_pred'][12910:] +\
                    model_2.params['ar_48_fit']/sum(model_2.params[1:])*data['ar_48_pred'][12910:]

In [234]:
MAPE_combined = mape_no_cv(combined_forecast, data['demand'][12910:])
MAE_combined = mae_no_cv(combined_forecast, data['demand'][12910:])

print(MAPE_combined,MAE_combined)

0.01563963666866373 20.20275923794491


### Model 1:
demand ~ ar_336_fit + nn_fit + ridge_fit

In [238]:
import statsmodels.formula.api as smf
formula = "demand ~ ar_336_fit + nn_fit + ridge_fit"
model_3 = smf.ols(formula = formula, data = data[:12910]).fit()
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:                 demand   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.254e+06
Date:                Mon, 29 Oct 2018   Prob (F-statistic):               0.00
Time:                        01:24:39   Log-Likelihood:                -55589.
No. Observations:               12910   AIC:                         1.112e+05
Df Residuals:                   12906   BIC:                         1.112e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.1615      0.707     22.875      0.0

In [246]:
# Get new weighted forecast.
combined_forecast = model_3.params['ar_336_fit']/sum(model_3.params[1:])*data['ar_336_pred'][12910:] + \
                    model_3.params['nn_fit']/sum(model_3.params[1:])* data['nn_pred'][12910:]+\
                    model_3.params['ridge_fit']/sum(model_3.params[1:])* data['ridge_pred'][12910:] 
            

In [247]:
MAPE_combined = mape_no_cv(combined_forecast, data['demand'][12910:])
MAE_combined = mae_no_cv(combined_forecast, data['demand'][12910:])

print(MAPE_combined,MAE_combined)

0.01539152327083829 19.831871071854202


### Model 2:
demand ~ ar_336_fit + nn_fit

In [251]:
formula = "demand ~ ar_336_fit + nn_fit"
model_4 = smf.ols(formula = formula, data = data[:12910]).fit()
print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:                 demand   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 1.690e+06
Date:                Mon, 29 Oct 2018   Prob (F-statistic):               0.00
Time:                        01:27:51   Log-Likelihood:                -56279.
No. Observations:               12910   AIC:                         1.126e+05
Df Residuals:                   12907   BIC:                         1.126e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     18.2574      0.743     24.572      0.0

In [252]:
# Get new weighted forecast.
combined_forecast = model_4.params['ar_336_fit']/sum(model_4.params[1:])*data['ar_336_pred'][12910:] + \
                    model_4.params['nn_fit']/sum(model_4.params[1:])* data['nn_pred'][12910:]
                   

In [253]:
MAPE_combined = mape_no_cv(combined_forecast, data['demand'][12910:])
MAE_combined = mae_no_cv(combined_forecast, data['demand'][12910:])

print(MAPE_combined,MAE_combined)

0.015435862803681831 19.928370175265147


#### COMBINE DATASET

In [217]:
ridge_res_train = pd.read_csv('../data/residuals_ridge_train_181028.csv')
del ridge_res_train['train_index']
ridge_res_test = pd.read_csv('../data/residuals_ridge_test_181028.csv')
del ridge_res_test['test_index']

ridge_res_test.Timestamp = pd.to_datetime(ridge_res_test.period,format='%Y-%m-%d %H:%M') 
ridge_res_test.index = ridge_res_test.Timestamp
ridge_res_test = ridge_res_test.sort_index()

ridge_res_train.Timestamp = pd.to_datetime(ridge_res_train.period,format='%Y-%m-%d %H:%M') 
ridge_res_train.index = ridge_res_train.Timestamp
ridge_res_train = ridge_res_train.sort_index()

In [218]:
nn_res_train = pd.read_csv('../data/residuals_nn_train_181028.csv')
del nn_res_train['train_index']
nn_res_test = pd.read_csv('../data/residuals_nn_test_181028.csv')
del nn_res_test['test_index']

nn_res_test.Timestamp = pd.to_datetime(nn_res_test.period,format='%Y-%m-%d %H:%M') 
nn_res_test.index = nn_res_test.Timestamp
nn_res_test = nn_res_test.sort_index()

nn_res_train.Timestamp = pd.to_datetime(nn_res_train.period,format='%Y-%m-%d %H:%M') 
nn_res_train.index = nn_res_train.Timestamp
nn_res_train = nn_res_train.sort_index()

In [219]:
nn_all = pd.concat([nn_res_train,nn_res_test])
ridge_all = pd.concat([ridge_res_train,ridge_res_test])

In [220]:
df = pd.read_csv('../data/shorter_clean_data_26_10_2018.csv',)
data = df[1632:]
data.Timestamp = pd.to_datetime(data.period,format='%Y-%m-%d %H:%M') 
data.index = data.Timestamp

In [221]:
AR_336_fit = pd.read_csv("../data/AR336_train_data.csv")
AR_336_result  = pd.read_csv('../data/AR_336_test_data.csv',)

In [222]:
AR_336_fitted = AR_336_fit['fit_AR_336'][1632:]
AR_336_fitted.index = nn_res_train.index
AR_336_fitted = AR_336_fitted+data['demand']

In [223]:
AR_336_pred = AR_336_result['pred_AR_336']
AR_336_pred.index = nn_res_test.index

In [224]:
AR_48_result = pd.read_csv('../data/SAR_test_data.csv',)
AR_48_fit = pd.read_csv("../data/SAR_train_data.csv")

AR_48_fitted = AR_48_fit['fit_AR_1'][1632:]
AR_48_fitted.index = nn_res_train.index
AR_48_fitted = AR_48_fitted+data['demand']

AR_48_pred = AR_48_result['AR_41_R_select']
AR_48_pred.index = nn_res_test.index

In [225]:
data['ar_336_fit'] = AR_336_fitted
data['ar_336_pred'] = AR_336_pred

data['ar_48_fit'] = AR_48_fitted
data['ar_48_pred'] = AR_48_pred

data['nn_fit'] = nn_res_train['predictions']
data['nn_pred'] = nn_res_test['predictions']

data['ridge_fit'] = ridge_res_train['predictions']
data['ridge_pred'] = ridge_res_test['predictions']

In [226]:
data.to_csv("../data/combine_model_result_10_29.csv")